<a href="https://colab.research.google.com/github/neurall/PlenoxelsColab/blob/main/Plenoxels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an attempt to make running in Colab cloud GPU possible for the wonderfull breakthrough that is Plenoxels made by fantastic team of incredibly talented  

# Plenoxels Authors: (Alex Yu * Sara Fridovich-Keil * Matthew Tancik Qinhong Chen Benjamin Recht Angjoo Kanazawa UC Berkeley * Equal contribution)  

Github:            https://github.com/sxyu/svox2  
Website and video: https://alexyu.net/plenoxels  
arXiv:             https://arxiv.org/abs/2112.05131  

Their achievement of shortening training from days to minutes is nothing short of extreaordinary. Incredible feat and wonderfull insight.

But due to recent shortage of gpus on market simple instant and easily accesible GPU cloud based workflow seems to be more important to fast research iteration than ever.
<br>

This first version seems to work at least on Colab Pro providing 16g gpu and with high mem option enabled(32g ram) to fit and train M60 dataset (Colab Pro is 10$ a month which is price of 1 hamburger)  
<br>
This colab will currently sadly not yet work on free colab (but deffinitely can soon, see idea proposed below?) instance due to size of datasets. And niether will it probably reasonably well work (if you reduce res it will fit in gpu mem but output typically will be unusable) on any gpu commonly at your pc like 10g 3080 or 11g 1080   

# Free colab has just 12g ram 11g (P8) gpu or 15g (t4?) gpu and this svox2 variant needs at least 27g ram and at least 16g gpu (p100?) for training datasets of similar size as cool M60 Tank dataset from my observations so far  

Hmm when you think about it These Larger datasets should be splitable to smaller ones fitting 4-24g gpus by estimated camera locations from bundle adjustment colmap files that datasets already have?  
<br>
Multi gpu (or multipass for free colab?) can than in theory work like this.  

Say you have 2 x 11g gpus. Split dataset photos to 2 x 2 chunks ABCD by estimated cam positions that datasets already have  
<br>
Then train on as many gpus in paralell as you want  
<br>
pass1 train individual scene fragments
<br>
gpu1:AB gpu2:CD  
<br>
pass2 to solve underrepresented voxels from border regions of chunks to allow good merge of seams 
<br>
gpu1:BC gpu2:DA    
<br>
merge pass1 and pass2  
<br>
Datasets:
NeRF-synthetic and front facing llff: 
https://drive.google.com/drive/folders/128yBriW1IG_3NJ5Rp7APSTZsJqdJdfc1  
Processed Tanks and temples dataset (with background): 
https://drive.google.com/file/d/1PD4oTP4F8jTtpjd_AQjCsL4h8iYFCyvO  
Real Lego capture: 
https://drive.google.com/file/d/1PG-KllCv4vSRPO7n5lpBjyTjlUyT8Nag  
Pretrained checkpoints (good if you cant train?): 
https://drive.google.com/drive/folders/1SOEJDw8mot7kf5viUK9XryOAmZGe_vvE  



In [ ]:
# Sadly. my 3080 while it more than twice as fast as Colab Pro is sitting iddle due to just 10g gpu mem
# So let's see what gpu colab given us (16g gpu mem is minumum for M60 tank sample dataset training and is given on Colab Pro instances).
from psutil import virtual_memory
gpu_info = !nvidia-smi 
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('BEWARE!!! Not connected to a GPU. Nothing will work')
else:
  print(gpu_info,'\n\nBEWARE!!! GPUs with less than 16g will fail with out of memory on M60 training near the end of training.')

ram_gb = virtual_memory().total / 1e9
print('\nYour runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('BEWARE!!! Not using a high-RAM runtime. M60 training will fail on out of general memory')
else:
  print('Great. You are using a high-RAM runtime! 27g is needed to train M60 \n')
!python --version

In [ ]:
#comment capture out to see cell outputs first time. I personally consider it unecessary super long distracting spam afterwards and cell execution done checkmark is all I need.
%%capture 
import os

%cd '/content'
if not os.path.exists('svox2'):
  !git clone https://github.com/sxyu/svox2.git  &> /dev/null

  # patch and make py sources again compatible with CoLab (python 3.7) by removing 3.8 specific syntax from two lines
  !sed -E -i "s/\{minv=:/minv=\{minv:/g"    /content/svox2/opt/opt.py 
  !sed -E -i "s/\{meanv=:/meanv=\{meanv:/g" /content/svox2/opt/opt.py 
  !sed -E -i "s/\{maxv=:/maxv=\{maxv:/g"    /content/svox2/opt/opt.py 
  !sed -E -i "s/\{minv=:/minv=\{minv:/g"    /content/svox2/opt/render_imgs.py 
  !sed -E -i "s/\{meanv=:/meanv=\{meanv:/g" /content/svox2/opt/render_imgs.py 
  !sed -E -i "s/\{maxv=:/maxv=\{maxv:/g"    /content/svox2/opt/render_imgs.py 
  
  # there is no point in slow conda and especiall this multi env dance in throwavay colab env anyway and most packages are installed already.
  # And if we stick to py 3.7 pytorch cudatoolkit and most of environment.yml etc. is already installed and ready here so install just 4 missing packages

  !pip install imageio-ffmpeg
  !pip install ipdb
  !pip install lpips
  !pip install pymcubes

  # this helps to lower gpu alloc waste and in turn allows fitting 16g gpu mem typically present in cloud gpus as max
  # ie I was last time 20mb short instead of 2g short on gpu mem
  %env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:21

%cd svox2

In [ ]:
# we can use gdrive to cache compiled pytorch wheel and checkpoint after training so next time colab runtime resets and forgets all files next bootstrap is fast
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# compile and install pytorch wheel is soooo sloooow. So we compile just once cache on gdrive and then just reinstall on each runtime restart
# we cache whl for each  gpu you encounter and save whls to gdrive in dirs with gpu names since they can and will vary
import os
from google.colab import files
%cd /content/svox2

# grab currenly assigned gpu model name (P100-16gb etc.)
gpuname = !nvidia-smi -L
gpuname = str(gpuname).replace('PCIE-','').split('Tesla ')[1].split(' ')[0]
whlname = 'svox2-0.0.1.dev0+sphtexcub.lincolor.fast-cp37-cp37m-linux_x86_64.whl'
whlpath = '/content/drive/MyDrive/'+gpuname+'/'+whlname
print(gpuname)

# compile this whl just once first time to obtain and cache it on gdrive
if not os.path.exists(whlpath):
  !apt install ninja-build
  %env MAX_JOBS=4
  !python setup.py bdist_wheel &> /dev/null
  !mkdir /content/drive/MyDrive/{gpuname}
  !cp ./dist/{whlname} {whlpath}

# install cached whl next time env is reset to skip costly recompilation
if os.path.exists(whlpath):
  !pip install {whlpath} --force-reinstall

In [ ]:
# download super cool tank dataset if not dir already present
%%capture 
%cd /content/svox2
import os
dataset_gdrive_ids={'TanksAndTempleBG':'1PD4oTP4F8jTtpjd_AQjCsL4h8iYFCyvO', # Datasets: from gdrive folder where id is last folder name in url: https://drive.google.com/drive/folders/128yBriW1IG_3NJ5Rp7APSTZsJqdJdfc1
                    'nerf_llff_data'  :'16VnMcF1KJYxN9QId6TClMsZRahHNMW5g',
                    'nerf_real_360'   :'1jzggQ7IPaJJTKx9yLASWHrX8dXHnG5eB',
                    'nerf_synthetic'  :'18JxhpWD-4ZmuFKLzKlAw-w5PpzZxXOcG',
                    'nerf_example'    :'1xzockqgkO-H3RCGfkZvIZNjOnk3l7AcT',
                    'lego_real_night_radial':'1PG-KllCv4vSRPO7n5lpBjyTjlUyT8Nag'}
dataset = 'TanksAndTempleBG'
if not os.path.exists(dataset):
  !gdown --id {dataset_gdrive_ids[dataset]}  &> /dev/null
  !tar -xvf {dataset}.tar.gz  &> /dev/null
  !rm  -rf  {dataset}.tar.gz

In [ ]:
# launch actual training (27g ram and 16g gpu for M60 dtataset needed)
%cd /content/svox2/opt
!./launch.sh m60 0 ../TanksAndTempleBG/M60 -c configs/tnt.json

In [ ]:
# since training runs detached we can peek at output on demand here when needed
# stop this manually when it ends thou!!! so next cells can run (look for " final save " text in output)
%cd /content/svox2/opt
!tail -f ckpt/m60/log

In [ ]:
# lets see our final resulting trained checkpoint file size when done . official one is 4G
!ls -la /content/svox2/opt/ckpt/m60/ckpt.npz

In [ ]:
# backup our expensively trained checkpoint to gdrive for future rendering tests if needed 
%%capture 
import os
%cd /content/svox2/opt
if os.path.exists('/content/svox2/opt/ckpt/m60/ckpt.npz'):
  !cp -R /content/svox2/opt/ckpt //content/drive/MyDrive/ckpt


In [ ]:
# (Sadly even thou colab pro has 16g it is still just 16g gpu)
# SO. If the damn training for too large datasets keeps failing with: out of memory even for cloud typically 12g or 16g gpus 
#
# Then. Luckily. We can still at least try download already trained checkpoints (thx. to paper authors ;D yay)
# And at least start playing with rendering images from various angles
#
# BUT BEWARE this is 11g gz download if enabled
# AND ALSO: if 3-4 downloads happen per 24h of this 11g large file from gdrive, which is highly plausible given usage in this colab. 
# Then google colab will block further downloads via this gdown api at this particular day for 24h
# If that happens. You can still supposedly download it via browser and curl/wget or upload it here by other means I guess?

%cd /content/svox2/opt
import os

# change this to True to enable pretrained checkpoints download. beware again 11g
if False:
  !mv ckpt ckpt_our
  if not os.path.exists('ckpt_tnt.tar.gz'): 
    !gdown --id 1v9xb5Sd3ulofwNUynC71I_fdwnSLnFhS 
  !tar -xvf ckpt_tnt.tar.gz  &> /dev/null
  !rm  -rf  ckpt_tnt.tar.gz # delete this huge 11g file as fast as possible
  !ls -la tnt_equirectlin_fasttv_autoscale/M60/ckpt.npz # check ckpt size for m60. should be around 4g
  !mv tnt_equirectlin_fasttv_autoscale ckpt

In [ ]:
# finally lets synthetize some sample images from various angles (typically 360 orbit around object) from trained checkpoint / model
%cd /content/svox2/opt
!python -u render_imgs.py ckpt/m60 ../TanksAndTempleBG/M60 

In [ ]:
# transfer 3m jpgs in zip is more practical than 40mb pngs or hard to inspect mp4 with too aggresive compression
%%capture 
%cd /content/svox2/opt/ckpt/m60/test_renders
!for i in *.png; do ffmpeg -i "$i" "${i%.*}.jpg" &> /dev/null ; done 
!find . -type f -iname \*.png -delete
%cd /content/svox2/opt
!zip -r m60.zip /content/svox2/opt/ckpt/m60/test_renders &> /dev/null 

In [ ]:
# copy resulting images to special in html cell visible dir 
%%capture 
!cp /content/svox2/opt/ckpt/m60/test_renders/*.* /usr/local/share/jupyter/nbextensions/ &> /dev/null
!cp m60.zip /usr/local/share/jupyter/nbextensions/ &> /dev/null

In [ ]:
# show some sample synthetized images
%%html
<a href src='/nbextensions/m60.zip' download>Download Images</a><br>
<img width="100%" src='/nbextensions/0000.jpg' />
<img width="100%" src='/nbextensions/0020.jpg' />
<img width="100%" src='/nbextensions/0031.jpg' />